In this homework assignment, you are requested to implement a full backprop algorithm using only *numpy*.

- We assume sigmoid activation across all layers.
- We assume a single value in the output layer

In [12]:
import numpy as np
np.random.seed(42)

The following class represents a simple feed forward network with multiple layers. The network class provides methods for running forward and backward for a single instance, throught the network. You should implement the methods (indicated with TODO), that performs forward and backward for an entire batch. Note, the idea is to use matrix multiplications, and not running standard loops over the instances in the batch.

In [13]:
np.random.seed(42)
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
from utils import MyNN

In [14]:
nn = MyNN(0.01, [3, 2, 1])

In [15]:
nn.model_params

{'W_1': array([[ 0.04967142, -0.01382643,  0.06476885],
        [ 0.15230299, -0.02341534, -0.0234137 ]]),
 'b_1': array([0.15792128, 0.07674347]),
 'W_2': array([[-0.04694744,  0.054256  ]]),
 'b_2': array([-0.04634177])}

In [16]:
x = np.random.randn(3)
y = np.random.randn(1)

y_hat = nn.forward_single_instance(x)
print(y_hat)

[0.48946]


In [17]:
nn.backward_single_instance(y)

2
1


In [18]:
def train(X, y, epochs, batch_size):
  '''
  Train procedure, please note the TODOs inside
  '''
  for e in range(1, epochs + 1):
    epoch_loss = 0
    # TODO: shuffle
    batches = np.random.permutation(X.shape[1])
    #... TODO: divide to batches
    batch_indices = np.array_split(batches, X.shape[1] // batch_size)
    
    for batch_idx in batch_indices:
        # Use indices to select the corresponding data
        X_b = X[:, batch_idx]
        y_b = y[:, batch_idx]
        
        y_hat = nn.forward_batch(X_b)
        epoch_loss += nn.log_loss_batch(y_hat, y_b)
        nn.backward_batch(y_b)
        nn.update()
    print(f'Epoch {e}, loss={epoch_loss/len(batch_indices)}')


In [19]:
# TODO: Make sure the following network trains properly

nn = MyNN(0.001, [6, 4, 3, 1])

X = np.random.randn(6, 100)
y = np.random.randn(1, 100)
batch_size = 8
epochs = 2

train(X, y, epochs, batch_size)

NameError: name 'nn' is not defined

#TODO: train on an external dataset

Train on the *hour.csv* file with a split of 75% training 10% validation and 15% for test.
Use the following features from the data:

* temp
* atemp
* hum
* windspeed
* weekday

The response variable is, *success*

The architecture of the network should be: [5, 40, 30, 10, 7, 5, 3, 1].

Use batch_size=8, and train it for 100 epochs on the train set (based on the split as requested above).

Then, plot train and validation loss per epoch.

In [ ]:
# TODO: Make sure the following network trains properly

nn = MyNN(0.001, [6, 4, 3, 1])

X = np.random.randn(6, 100)
y = np.random.randn(1, 100)
batch_size = 8
epochs = 2

train(X, y, epochs, batch_size)

3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1


ValueError: operands could not be broadcast together with shapes (4,) (6,) 

##  your code goes here

###  Data Preprocessing

In [ ]:
class DataLoader:
  def __init__(self, X, y, batch_size):
    self.X = X
    self.y = y
    self.batch_size = batch_size
    self.indices = np.random.permutation(X.shape[1])
    self.batches = np.array_split(self.indices, self.batch_size)

  def __iter__(self):
    return self
  
  def __next__(self):
    if len(self.batches) == 0:
      raise StopIteration
    batch_indices = self.batches.pop(0)
    X_batch = self.X[:, batch_indices]
    y_batch = self.y[batch_indices]
    return X_batch, y_batch

def preprocess_data(file_path):
  '''
  Preprocess the bike sharing dataset ('hour.csv')
  '''
  # Load numpy arrays
  with open(file_path, 'r') as file:
    data = np.loadtxt(file, delimiter=',', skiprows=1)  # skiprows=1 to skip the header row
    # Select required features: temp, atemp, hum, windspeed, weekday
    X = data[:, [10, 11, 12, 13, 7]]  # indices for temp, atemp, hum, windspeed, weekday
    y = data[:, 17]  # index for success
    # Normalize/standardize features
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Split data into training, validation, and test sets
    train_X, train_y, val_X, val_y, test_X, test_y = split_data(X, y)
    # Create DataLoader objects
    train_loader = DataLoader(train_X, train_y, batch_size=8)
    val_loader = DataLoader(val_X, val_y, batch_size=8)
    test_loader = DataLoader(test_X, test_y, batch_size=8)
    return train_loader, val_loader, test_loader


def split_data(X, y, train_ratio=0.75, val_ratio=0.1, test_ratio=0.15):
  '''
  Split the data into training, validation, and test sets
  X - numpy array of shape (num_instances, num_features)
  y - numpy array of shape (num_instances,)
  train_ratio - float, the ratio of the training set
  val_ratio - float, the ratio of the validation set
  test_ratio - float, the ratio of the test set
  Returns:
  train_X, train_y, val_X, val_y, test_X, test_y
  '''
  # Calculate the number of instances for each set
  num_instances = X.shape[0]
  num_train = int(train_ratio * num_instances)
  num_val = int(val_ratio * num_instances)

  # Shuffle the data
  indices = np.random.permutation(num_instances)
  X = X[indices]
  y = y[indices]

  # Split the data
  train_X = X[:num_train]
  train_y = y[:num_train]
  val_X = X[num_train:num_train+num_val]
  val_y = y[num_train:num_train+num_val]
  test_X = X[num_train+num_val:]
  test_y = y[num_train+num_val:]

  return train_X, train_y, val_X, val_y, test_X, test_y

### Model Training


In [ ]:
# TODO: Create visualizations of the learning process
# - Plot the training loss per epoch
# - Create additional relevant plots (validation loss, learning curves, etc.)
# - Make sure all plots have proper labels, titles, and legends
# - Add brief analysis of what the plots reveal about your model's performances

class TrainingVisualizer:
    def __init__(self):
        """Initialize the training visualizer with empty history."""
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_accuracy': [],
            'val_accuracy': [],
            'learning_rates': []
        }
    
    def update(self, 
              val_loss: float, 
              train_loss: float, 
              train_accuracy: float = None, 
              val_accuracy: float = None,
              learning_rate: float = None):
        """Update the training history with new metrics."""
        self.history['train_loss'].append(train_loss)
        self.history['val_loss'].append(val_loss)
        if train_accuracy is not None:
            self.history['train_accuracy'].append(train_accuracy)
        if val_accuracy is not None:
            self.history['val_accuracy'].append(val_accuracy)
        if learning_rate is not None:
            self.history['learning_rates'].append(learning_rate)
    
    def plot_training_curves(self, save_path: str = None):
        """Plot training and validation loss curves."""
        plt.figure(figsize=(12, 5))
        
        # Plot loss curves
        plt.subplot(1, 2, 1)
        plt.plot(self.history['train_loss'], label='Training Loss', color='blue')
        plt.plot(self.history['val_loss'], label='Validation Loss', color='red')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        
        # Plot accuracy curves if available
        if self.history['train_accuracy'] and self.history['val_accuracy']:
            plt.subplot(1, 2, 2)
            plt.plot(self.history['train_accuracy'], label='Training Accuracy', color='blue')
            plt.plot(self.history['val_accuracy'], label='Validation Accuracy', color='red')
            plt.title('Training and Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.grid(True)
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()
    
    def plot_learning_rate(self, save_path: str = None):
        """Plot learning rate changes over time."""
        if not self.history['learning_rates']:
            return
        
        plt.figure(figsize=(8, 4))
        plt.plot(self.history['learning_rates'], color='green')
        plt.title('Learning Rate Changes')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.grid(True)
        
        if save_path:
            plt.savefig(save_path)
        plt.show()
    
    def plot_loss_distribution(self, save_path: str = None):
        """Plot the distribution of training and validation losses."""
        plt.figure(figsize=(10, 4))
        
        # Plot histograms
        plt.subplot(1, 2, 1)
        plt.hist(self.history['train_loss'], bins=30, alpha=0.5, label='Training Loss', color='blue')
        plt.hist(self.history['val_loss'], bins=30, alpha=0.5, label='Validation Loss', color='red')
        plt.title('Loss Distribution')
        plt.xlabel('Loss Value')
        plt.ylabel('Frequency')
        plt.legend()
        
        # Plot box plots
        plt.subplot(1, 2, 2)
        plt.boxplot([self.history['train_loss'], self.history['val_loss']], 
                   labels=['Training Loss', 'Validation Loss'])
        plt.title('Loss Box Plot')
        plt.ylabel('Loss Value')
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()
    
    def generate_summary(self) -> Dict:
        """Generate a summary of the training metrics."""
        summary = {
            'final_train_loss': self.history['train_loss'][-1],
            'final_val_loss': self.history['val_loss'][-1],
            'min_train_loss': min(self.history['train_loss']),
            'min_val_loss': min(self.history['val_loss']),
            'epochs_trained': len(self.history['train_loss'])
        }
        
        if self.history['train_accuracy'] and self.history['val_accuracy']:
            summary.update({
                'final_train_accuracy': self.history['train_accuracy'][-1],
                'final_val_accuracy': self.history['val_accuracy'][-1],
                'max_train_accuracy': max(self.history['train_accuracy']),
                'max_val_accuracy': max(self.history['val_accuracy'])
            })
        
        return summary

In [ ]:
# TODO: Train the neural network
# - Implement the network with architecture [5, 40, 30, 10, 7, 5, 3, 1]
# - Train for exactly 100 epochs on the training set
# - Use batch_size=8 as specified
# - Calculate and store train and validation loss for each epoch
# - Track training progres
def train_nn(train_loader: DataLoader, val_loader: DataLoader, epochs: int):
    """
    Train the neural network with visualization support.
    
    Parameters:
    train_loader: DataLoader object for the training set
    val_loader: DataLoader object for the validation set
    epochs: int, the number of epochs to train the network
    """
    nn = MyNN(0.01, [5, 40, 30, 10, 7, 5, 3, 1])
    visualizer = TrainingVisualizer()
    
    for epoch in range(epochs):
        # Training phase
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for X_batch, y_batch in train_loader:
            # Forward pass
            y_hat = nn.forward_batch(X_batch)
            train_loss += nn.log_loss_batch(y_hat, y_batch)
            
            # Calculate accuracy
            predictions = (y_hat > 0.5).astype(int)
            train_correct += np.sum(predictions == y_batch)
            train_total += y_batch.size
            
            # Backward pass and update
            nn.backward_batch(y_batch)
            nn.update()
        
        # Calculate average training loss and accuracy
        train_loss /= len(train_loader)
        train_accuracy = train_correct / train_total
        
        # Validation phase
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        for X_batch, y_batch in val_loader:
            y_hat = nn.forward_batch(X_batch)
            val_loss += nn.log_loss_batch(y_hat, y_batch)
            
            # Calculate accuracy
            predictions = (y_hat > 0.5).astype(int)
            val_correct += np.sum(predictions == y_batch)
            val_total += y_batch.size
        
        # Calculate average validation loss and accuracy
        val_loss /= len(val_loader)
        val_accuracy = val_correct / val_total
        
        # Update visualizer
        visualizer.update(
            train_loss=train_loss,
            val_loss=val_loss,
            train_accuracy=train_accuracy,
            val_accuracy=val_accuracy,
            learning_rate=nn.learning_rate
        )
        
        # Print progress
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    # Generate visualizations
    visualizer.plot_training_curves()
    visualizer.plot_learning_rate()
    visualizer.plot_loss_distribution()
    
    # Print training summary
    summary = visualizer.generate_summary()
    print("\nTraining Summary:")
    for metric, value in summary.items():
        print(f"{metric}: {value:.4f}")
    
    return nn, visualizer

### Visualization

### Model Evaluation


In [ ]:
# TODO: Evaluate model performance on the test set
# - Calculate and report the loss on the test set
# - Calculate and report the accuracy on the test set
# - Compare test performance with training/validation performance
# - Analyze model strengths and weaknesses
# - Discuss any overfitting/underfitting issues observed